# K-Prototypes Tech Talk
# By Palermo Penano

The summary below closely follows the original paper, [Huang 1998](https://link.springer.com/article/10.1023/A:1009769707641).

## Outline
* Unsupervised machine learning and what does it mean to cluster data
    * How does regular K-means work?
    * Other clustering algos
* What are its advantages and limitations
* Why is using standard distance metric problematic?
* K-prototypes
* What libraries are available?
* Application to Colombia vulnerability index

## References

* Python implementation
    * https://pypi.org/project/kmodes/
    * https://github.com/nicodv/kmodes
* Other articles and notebooks using k-prototypes
    * https://www.kaggle.com/rohanadagouda/unsupervised-learning-using-k-prototype-and-dbscan
    * https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

* Source for this demo
    * Dataset
        * https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/
    * Analysis
        * https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677
        * https://www.kaggle.com/paulinan/bank-customer-segmentation

* [Unsupervised clustering with mixed categorical and continuous data](https://www.tomasbeuzen.com/post/clustering-mixed-data/)
    
https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering







## How does K-means work

### K-means algorithm
* Initialize with a random set of cluster vectors (these are m-dimensional vectors where m is the number of features in the dataset)
* In a loop:
    * Allocate each point in the data to the nearest cluster based on the Euclidean distance
    * For each cluster, update the cluster vector by calculating the mean of all the records assigned to that cluster (average of each component across all vectors belonging to the same cluster).
    * Repeat the first step by checking and reallocating each data point based on the new cluster vectors
    * Repeat until desired number of iteration is reached or record changes cluster
    
Here's a visualization of the algorithm
    
    
![kmeans_algo](./imgs/kmeans_algo.png)
[Source](https://en.wikipedia.org/wiki/K-means_clustering)

## Why K-means shouldn't be used with categorical variables

Categorical variables are data that takes one of a limited and usually fixed number of possible values. If they can be ordered, then it is called `ordinal`. If it cannot be ordered, then it is called `nominal`. Here are some examples:

    * ordinal: risk rating, rank, day of the week
    * nominal: political party, region of residence, occupation

There are several ways these variables are used in models. For unsupervised ML, some approaches include one-hot encoding, label encoding, feature aggregation, or excluding the feature from the model.

For one-hot encoding, the categorical is converted to several binary features each of which representing a given category in the original feature. This works when there are few categories, but become a computational burden when categories exceeds hundres or thousands of unique values (e.g. zip codes).

Label encoding only works for ordinal data, but even in such case it would require that the encoded feature follows the same ordering as the original values in the raw feature.

In the case of a categorical feature with high cardinality, one solution is to aggregate them. For example, zip codes can be aggregated to a region containing many zip codes. Following the aggregation, one can then apply one-hot encoding.

When the approaches above fails or is infeasible to apply, the data scientist may exclude categorical feature all together from the model.

## Why is using standard distance metric problematic for categorical data?


For numerical data, it is natural to define the center of mass for a collection of points using common distance metrics, such as the Euclidean distance. Say you have three points in your dataset containing annual income: `[$100k, $45k, $65k]`. One metric for the center of mass is just the average, in this case, `$70k`. 

But instead of income, suppose you had categories of hair color: `[blond, red, black]`. How can one define center of mass in this case?


## K-prototypes

K-prototypes combines k-means and k-modes into to be able to handle both numeric and cateogircal variables. The k-modes algorithm differs from k-means in the following way:

- Instead of Euclidean distance, use an alternative dissimilarity measure for categorical objects
- Instead of mean of a set of points to define a cluster centroid, use the mode where a frequency-based approach to find modes for the clusters


### K-modes

Dissimilarity measure for categorical attributes

```
x1 = [finance, married, golf] 
	is similar to 
x2 = [finance, married, squash] 
	but dissimilar to 
x3 = [healthcare, single, soccer]
```

More formally,

<img src="./imgs/dissimilarity_measure.png" alt="drawing" width="250"/>

Use the mode to represent the "center" of a set of vectors containing only categoricals. The mode itself need not be part of the original set. The formal definition for the mode is a vector Q such that this Q minimizes the distance between itself and the other vectors in the set using the dissimilarity measure above.

More formally

<img src="./imgs/cat_mode.png" alt="drawing" width="500"/>

Now that we have an approach for defining how close two vectors contaning categorical values are and a technique for determining the center of mass for a set of categorical-valued vectors, we can use a similar algorithm as k-means to find the clusters.

The k-modes algorithm:

* Select the k initial modes, one for each cluster
* In a loop, 
    * Allocate each point in the data to the nearest cluster based on the dissimilarity measure defined above
    * Update the mode of the cluster based on the set of points allocated to it
    * Repeat the first step by checking and reallocating each data point based on the new cluster modes
    * Repeat until no record changes cluster or the desired number of iteration is reached

### Combining k-means and k-modes

Combining k-means and k-modes requires a function that combines both the Euclidean distance metric and the dissimilarity measure function. Formally, the function is formulated as 

<img src="./imgs/combined_euc_dissm.png" alt="drawing" width="300"/>

Similar to the k-means and k-modes algorithm above, the optimization problems boils down to finding the cluster vectors (that now contain both numeric and categorical values) such that the sum of the euclidean distance metric and dissimilarity measure across all k cluster vectors is minimized.

In [25]:
import warnings
from typing import List, Tuple

import numpy as np
import pandas as pd 
import random
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from scipy import stats
from matplotlib import style

from kmodes.kprototypes import KPrototypes
from kmodes.util import dissim               # Import module containing dissimilarity functions for categoricals
                                             # For list of all dissimilarity functions, see https://github.com/nicodv/kmodes/blob/master/kmodes/util/dissim.py

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.3f}'.format

# Clustering German Credit Data

In [2]:
col_names = [
    "chk_acct",          # 1
    "duration",          # 2
    "credit_his",        # 3
    "purpose",           # 4
    "amount",            # 5
    "saving_acct",       # 6
    "present_emp",       # 7
    "installment_rate",  # 8
    "sex",               # 9
    "other_debtor",      # 10          
    "present_resid",     # 11
    "property",          # 12
    "age",               # 13
    "other_install",     # 14
    "housing",           # 15
    "n_credits",         # 16
    "job",               # 17
    "n_people",          # 18
    "telephone",         # 19
    "foreign",           # 20
    "response"           # 21
]

df = pd.read_csv('./data/german/german.data',
                 sep=' ', 
                 header=None,
                 names=col_names)

In [3]:
# Restrict to this list of variables to simplify problem
# See https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677

cont_cols = [
    "duration",          # 2
    "amount",            # 5
    "installment_rate",  # 8
    "present_resid",     # 11
    "age",               # 13
    "n_credits",         # 16
    "n_people"           # 18
]
cat_cols = [
    "chk_acct",          # 1
    "credit_his",        # 3
    "present_emp",       # 7
    "sex",               # 9
    "property",          # 12
    "housing"            # 15
]

In [4]:
df = df.loc[:, cont_cols+cat_cols]

In [28]:
df.head().T

,0,1,2,3,4
duration,-1.236,2.248,-0.739,1.750,0.257
amount,-0.745,0.950,-0.417,1.634,0.567
installment_rate,0.918,-0.870,-0.870,-0.870,0.024
present_resid,1.047,-0.766,0.141,1.047,1.047
age,2.766,-1.191,1.183,0.832,1.535
n_credits,1.027,-0.705,-0.705,-0.705,1.027
n_people,-0.428,-0.428,2.335,2.335,2.335
chk_acct,A11,A12,A14,A11,A11
credit_his,A34,A32,A34,A32,A33
present_emp,A75,A73,A74,A74,A73


In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
duration            1000 non-null int64
amount              1000 non-null int64
installment_rate    1000 non-null int64
present_resid       1000 non-null int64
age                 1000 non-null int64
n_credits           1000 non-null int64
n_people            1000 non-null int64
chk_acct            1000 non-null object
credit_his          1000 non-null object
present_emp         1000 non-null object
sex                 1000 non-null object
property            1000 non-null object
housing             1000 non-null object
dtypes: int64(7), object(6)
memory usage: 101.7+ KB


In [6]:
df[cont_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",20.903,12.059,4.000,12.000,18.000,24.000,72.000
amount,"1,000.000","3,271.258","2,822.737",250.000,"1,365.500","2,319.500","3,972.250","18,424.000"
installment_rate,"1,000.000",2.973,1.119,1.000,2.000,3.000,4.000,4.000
present_resid,"1,000.000",2.845,1.104,1.000,2.000,3.000,4.000,4.000
age,"1,000.000",35.546,11.375,19.000,27.000,33.000,42.000,75.000
n_credits,"1,000.000",1.407,0.578,1.000,1.000,1.000,2.000,4.000
n_people,"1,000.000",1.155,0.362,1.000,1.000,1.000,1.000,2.000


In [7]:
def summarize_cats(df: pd.DataFrame, cat_cols: List[str] = []) -> pd.DataFrame:
    '''Create table summarizing categorical variables
    To display more values in Values col, set
    pd.set_option('display.max_colwidth', 100)
    '''

    df_cats = df.loc[:, cat_cols]

    print(f"Dataset Shape: {df_cats.shape}")
    summary = pd.DataFrame(df_cats.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Column Name'] = summary['index']
    summary = summary[['Column Name', 'dtypes']]
    summary['Missing'] = df_cats.isnull().sum().values
    summary['Uniques'] = df_cats.nunique().values

    for name in summary['Column Name'].value_counts().index:

        # List unique values
        list_uniques = [str(v) for v in df_cats[name].unique()]
        summary.loc[summary['Column Name'] == name,
                    'Values'] = ' | '.join(list_uniques)

        # Calculate entropy
        shares = df_cats[name].value_counts(normalize=True)
        summary.loc[summary['Column Name'] == name, 'Entropy'] = round(
            stats.entropy(shares, base=2), 2)

    return summary

In [8]:
summarize_cats(df, cat_cols=cat_cols)

Dataset Shape: (1000, 6)


,Column Name,dtypes,Missing,Uniques,Values,Entropy
0,chk_acct,object,0,4,A11 | A12 | A14 | A13,1.800
1,credit_his,object,0,5,A34 | A32 | A33 | A30 | A31,1.710
2,present_emp,object,0,5,A75 | A73 | A74 | A71 | A72,2.160
3,sex,object,0,4,A93 | A92 | A91 | A94,1.530
4,property,object,0,4,A121 | A122 | A124 | A123,1.950
5,housing,object,0,3,A152 | A153 | A151,1.140


,0,1,2,3,4
duration,-1.236,2.248,-0.739,1.750,0.257
amount,-0.745,0.950,-0.417,1.634,0.567
installment_rate,0.918,-0.870,-0.870,-0.870,0.024
present_resid,1.047,-0.766,0.141,1.047,1.047
age,2.766,-1.191,1.183,0.832,1.535
n_credits,1.027,-0.705,-0.705,-0.705,1.027
n_people,-0.428,-0.428,2.335,2.335,2.335
chk_acct,A11,A12,A14,A11,A11
credit_his,A34,A32,A34,A32,A33
present_emp,A75,A73,A74,A74,A73


# Clustering using K-Prototypes

In [10]:
# Scale numerical features using Z-scores

display(df[cont_cols].describe().T)
scaler = StandardScaler()
df.loc[:, cont_cols] = scaler.fit_transform(df.loc[:, cont_cols])
display(df[cont_cols].describe().T)

,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",20.903,12.059,4.000,12.000,18.000,24.000,72.000
amount,"1,000.000","3,271.258","2,822.737",250.000,"1,365.500","2,319.500","3,972.250","18,424.000"
installment_rate,"1,000.000",2.973,1.119,1.000,2.000,3.000,4.000,4.000
present_resid,"1,000.000",2.845,1.104,1.000,2.000,3.000,4.000,4.000
age,"1,000.000",35.546,11.375,19.000,27.000,33.000,42.000,75.000
n_credits,"1,000.000",1.407,0.578,1.000,1.000,1.000,2.000,4.000
n_people,"1,000.000",1.155,0.362,1.000,1.000,1.000,1.000,2.000


,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",0.000,1.001,-1.402,-0.739,-0.241,0.257,4.239
amount,"1,000.000",0.000,1.001,-1.071,-0.675,-0.337,0.248,5.371
installment_rate,"1,000.000",0.000,1.001,-1.765,-0.870,0.024,0.918,0.918
present_resid,"1,000.000",-0.000,1.001,-1.672,-0.766,0.141,1.047,1.047
age,"1,000.000",0.000,1.001,-1.455,-0.752,-0.224,0.568,3.470
n_credits,"1,000.000",-0.000,1.001,-0.705,-0.705,-0.705,1.027,4.491
n_people,"1,000.000",-0.000,1.001,-0.428,-0.428,-0.428,-0.428,2.335


In [14]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
duration            1000 non-null float64
amount              1000 non-null float64
installment_rate    1000 non-null float64
present_resid       1000 non-null float64
age                 1000 non-null float64
n_credits           1000 non-null float64
n_people            1000 non-null float64
chk_acct            1000 non-null object
credit_his          1000 non-null object
present_emp         1000 non-null object
sex                 1000 non-null object
property            1000 non-null object
housing             1000 non-null object
dtypes: float64(7), object(6)
memory usage: 101.7+ KB


In [12]:
# Get index of categorical columns
cat_cols_idx = [df.columns.get_loc(c) for c in cat_cols if c in df]

In [26]:
# Fit and predict kprototypes

# For more details on parameters, see source code:
# https://github.com/nicodv/kmodes/blob/master/kmodes/kprototypes.py#L335

kp = KPrototypes(
        n_clusters=5,
        max_iter=100,                      # Number of iteration (cluster updates) per run (n_init)
        cat_dissim=dissim.matching_dissim, # Dissimilarity function to use for cat. features
        init='Cao',                        # Initialization method (see orignal paper for details)
        n_init=10,                         # Number of time the k-modes algorithm will be run with different centroid seeds.
        gamma=None,                        # Weighing factor that determines relative importance of cont. vs cat. features
        verbose=300,                       # Verbosity of output during training
        random_state=None,
        n_jobs=-1,
)

clusters = kp.fit_predict(df, categorical=cat_cols_idx)

Best run was number 3


In [31]:
# Output is an array with cluster assignment for each record in original dataset
display(clusters.shape)
display(clusters)

(1000,)

array([2, 1, 4, 1, 4, 1, 2, 1, 2, 3, 3, 1, 0, 2, 0, 3, 2, 1, 1, 4, 2, 4,
       4, 2, 0, 0, 2, 2, 3, 1, 3, 0, 0, 2, 3, 1, 1, 3, 0, 3, 3, 3, 4, 0,
       2, 2, 3, 0, 0, 3, 3, 0, 3, 0, 2, 4, 0, 1, 3, 1, 0, 2, 2, 1, 3, 4,
       0, 3, 3, 3, 1, 2, 4, 1, 2, 2, 3, 2, 1, 0, 0, 2, 3, 2, 2, 3, 3, 1,
       4, 3, 2, 2, 4, 3, 2, 1, 2, 3, 2, 4, 2, 3, 3, 3, 0, 1, 4, 0, 0, 0,
       4, 3, 3, 1, 3, 1, 1, 0, 0, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 1, 1,
       0, 2, 1, 3, 1, 2, 3, 0, 0, 3, 3, 0, 3, 1, 0, 2, 1, 2, 4, 0, 3, 1,
       1, 0, 4, 3, 0, 4, 4, 3, 3, 2, 3, 0, 3, 3, 0, 2, 4, 3, 3, 3, 0, 1,
       0, 2, 4, 2, 1, 1, 4, 2, 4, 3, 4, 2, 3, 0, 2, 4, 4, 0, 3, 0, 0, 2,
       3, 3, 2, 3, 3, 3, 4, 4, 2, 3, 0, 2, 4, 2, 2, 2, 2, 0, 3, 0, 0, 2,
       3, 3, 3, 4, 3, 3, 1, 2, 3, 3, 3, 2, 3, 3, 4, 4, 1, 2, 3, 4, 3, 4,
       4, 3, 4, 3, 2, 3, 0, 3, 2, 0, 3, 0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 2,
       4, 4, 1, 3, 0, 3, 3, 2, 1, 1, 1, 4, 0, 2, 0, 3, 3, 3, 2, 2, 3, 1,
       4, 1, 2, 2, 0, 1, 4, 2, 1, 1, 0, 3, 2, 3, 2,

In [33]:
df['clusters'] = clusters

In [38]:
# Continuous variablesby clusters
df.groupby('clusters')[cont_cols].describe().T

clusters                     0       1       2       3       4
duration         count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.377   1.663  -0.298  -0.175  -0.253
                 std     0.633   0.930   0.737   0.678   0.897
                 min    -1.236  -1.236  -1.319  -1.236  -1.402
                 25%    -0.739   1.253  -0.739  -0.739  -0.905
                 50%    -0.490   1.501  -0.490  -0.241  -0.490
                 75%     0.257   2.248   0.257   0.257   0.257
                 max     2.248   4.239   2.248   1.999   2.248
amount           count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.134   1.924  -0.357  -0.419  -0.172
                 std     0.561   1.092   0.580   0.441   0.654
                 min    -1.071  -0.078  -1.040  -1.062  -1.006
                 25%    -0.542   1.175  -0.739  -0.723  -0.678
                 50%    -0.232   1.669  -0.542  -0.515  -0.306
                 75%     0.134   2.534  -0.167  -0.209   0.116
                 max     2.023   5.371   1.628   1.172   2.606
installment_rate count 200.000 135.000 179.000 346.000 140.000
                 mean   -1.232  -0.300   0.339   0.714  -0.148
                 std     0.440   0.963   0.829   0.376   1.040
                 min    -1.765  -1.765  -1.765   0.024  -1.765
                 25%    -1.765  -0.870   0.024   0.918  -0.870
                 50%    -0.870  -0.870   0.918   0.918   0.024
                 75%    -0.870   0.918   0.918   0.918   0.918
                 max    -0.870   0.918   0.918   0.918   0.918
present_resid    count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.181  -0.048   0.819  -0.334   0.082
                 std     1.024   0.991   0.507   0.959   0.968
                 min    -1.672  -1.672  -0.766  -1.672  -1.672
                 25%    -0.766  -0.766   1.047  -0.766  -0.766
                 50%    -0.766   0.141   1.047  -0.766   0.141
                 75%     1.047   1.047   1.047   0.820   1.047
                 max     1.047   1.047   1.047   1.047   1.047
age              count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.512  -0.003   1.345  -0.510   0.276
                 std     0.632   0.906   0.914   0.548   0.765
                 min    -1.455  -1.279  -0.752  -1.367  -1.367
                 25%    -1.015  -0.708   0.656  -0.928  -0.224
                 50%    -0.664  -0.224   1.271  -0.664   0.216
                 75%    -0.202   0.568   2.063  -0.136   0.656
                 max     1.799   2.854   3.470   1.623   3.382
n_credits        count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.151   0.014   0.369  -0.214   0.260
                 std     0.864   0.979   1.191   0.803   1.180
                 min    -0.705  -0.705  -0.705  -0.705  -0.705
                 25%    -0.705  -0.705  -0.705  -0.705  -0.705
                 50%    -0.705  -0.705   1.027  -0.705  -0.705
                 75%     1.027   1.027   1.027   1.027   1.027
                 max     2.759   4.491   4.491   2.759   4.491
n_people         count 200.000 135.000 179.000 346.000 140.000
                 mean   -0.428  -0.121  -0.428  -0.428   2.335
                 std     0.000   0.872   0.000   0.000   0.000
                 min    -0.428  -0.428  -0.428  -0.428   2.335
                 25%    -0.428  -0.428  -0.428  -0.428   2.335
                 50%    -0.428  -0.428  -0.428  -0.428   2.335
                 75%    -0.428  -0.428  -0.428  -0.428   2.335
                 max    -0.428   2.335  -0.428  -0.428   2.335

In [72]:
for c in cat_cols:

    # Get counts across clusters and categorical column
    table = pd.pivot_table(
        df, index=["clusters"], columns=[c],
        aggfunc=len, margins=True, dropna=True, fill_value=0
    )

    # Divide by total across all cateogories for a given cluster
    table2 = table.div(table.iloc[:, -1], axis=0).iloc[:-1,:-1]

    # Apply background gradient to generate a heatmap
    display(table2.style.background_gradient(cmap='Blues', axis=1))

chk_acct,A11,A12,A13,A14
clusters,,,,
0,0.26,0.295,0.085,0.36
1,0.237037,0.407407,0.00740741,0.348148
2,0.22905,0.212291,0.0893855,0.469274
3,0.291908,0.260116,0.0578035,0.390173
4,0.342857,0.192857,0.0642857,0.4


credit_his,A30,A31,A32,A33,A34
clusters,,,,,
0,0.045,0.035,0.615,0.08,0.225
1,0.0814815,0.0592593,0.42963,0.17037,0.259259
2,0.027933,0.0167598,0.374302,0.0726257,0.50838
3,0.0289017,0.0520231,0.635838,0.066474,0.216763
4,0.0357143,0.0928571,0.442857,0.0928571,0.335714


present_emp,A71,A72,A73,A74,A75
clusters,,,,,
0,0.065,0.22,0.4,0.18,0.135
1,0.0814815,0.140741,0.311111,0.251852,0.214815
2,0.0949721,0.0446927,0.212291,0.106145,0.541899
3,0.0433526,0.234104,0.404624,0.17052,0.147399
4,0.0428571,0.142857,0.278571,0.185714,0.35


sex,A91,A92,A93,A94
clusters,,,,
0,0.06,0.465,0.335,0.14
1,0.0666667,0.237037,0.666667,0.0296296
2,0.0614525,0.251397,0.614525,0.0726257
3,0.0462428,0.364162,0.459538,0.130058
4,0.0142857,0.1,0.871429,0.0142857


property,A121,A122,A123,A124
clusters,,,,
0,0.265,0.25,0.415,0.07
1,0.103704,0.177778,0.385185,0.333333
2,0.351955,0.234637,0.234637,0.178771
3,0.303468,0.245665,0.349711,0.101156
4,0.335714,0.221429,0.242857,0.2


housing,A151,A152,A153
clusters,,,
0,0.285,0.69,0.025
1,0.148148,0.614815,0.237037
2,0.117318,0.709497,0.173184
3,0.179191,0.771676,0.0491329
4,0.135714,0.7,0.164286


In [ ]:
- cluster means
- do the clusters represent distribution of categoricals?

# Cluster centroids

In [34]:
kp.cluster_centroids_

array([['-0.3769253787636637', '-0.1341507905942031',
        '-1.2323870856776291', '-0.18129639453156568',
        '-0.5119712341207943', '-0.15068442717407948',
        '-0.4282895663715406', 'A14', 'A32', 'A73', 'A92', 'A123',
        'A152'],
       ['1.6631135941260122', '1.924092350533361',
        '-0.30046184094451744', '-0.04750637004854951',
        '-0.0030685633914357244', '0.013535297630366675',
        '-0.1212719560693609', 'A12', 'A32', 'A73', 'A93', 'A123',
        'A152'],
       ['-0.2978690214456571', '-0.35656841203208295',
        '0.33891119500681627', '0.8191001489806753',
        '1.3449679422481038', '0.36911056594167', '-0.4282895663715404',
        'A14', 'A34', 'A75', 'A93', 'A121', 'A152'],
       ['-0.17467435071668727', '-0.4193149783055801',
        '0.7142803762475702', '-0.33369540132204656',
        '-0.5104079173824942', '-0.21435813369372306',
        '-0.428289566371541', 'A14', 'A32', 'A73', 'A93', 'A123', 'A152'],
       ['-0.252709851910948', 

In [ ]:
kp.cluster_centroids_[0]